In [ ]:
# simple online real time tracking


#TensorFlow 2 Object Detection API with Google Colab
Author: Nisarg Kapkar

Link https://medium.com/swlh/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc?source=friends_link&sk=0bb205df0e1c29a2e78c28671ddf4494

Sử dụng NoteBook này cùng với bài viết Phương tiện được đề cập, bài viết chứa thông tin chi tiết (không được đề cập trong NoteBook này) cho Bước 1, Bước 2 và Bước 13!

Liên kết đến Kho lưu trữ GitHub!

Bước 1- Điều kiện tiên quyết (Thu thập / Gắn nhãn hình ảnh, Tạo bản đồ nhãn ...)

Tham khảo link bài viết được đề cập để biết thêm chi tiết!

Bước 2- Thiết lập cấu trúc thư mục trên Google Drive.

Tham khảo link bài viết được đề cập để biết thêm chi tiết!

Bước 3- Chọn Bộ tăng tốc phần cứng.

Trên Colab, đi tới Runtime-> Change Runtime Type và chọn Trình tăng tốc phần cứng làm GPU.

LƯU Ý: Nếu bạn đã đặt các tên khác nhau cho tệp và thư mục của mình, hãy thay đổi đường dẫn trong các ô bên dưới theo tệp và thư mục của bạn!

#Imports and Setup

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!nvidia-smi

Thu Oct 29 07:34:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

mount google drive:

In [ ]:
#Step 4- Mount Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Step 5- Download TensorFlow Model Garden.

#cd into the TensorFlow directory in your Google Drive
%cd '/content/drive/My Drive/TensorFlow'

#and clone the TensorFlow Model Garden repository
!git clone https://github.com/tensorflow/models.git

In [ ]:
#Step 6- Install some required libraries and tools.

!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim

In [ ]:
#Step 7- Compile the Protobuf libraries.

#cd into 'TensorFlow/models/research'
%cd '/content/drive/My Drive/TensorFlow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
#Step 8- Set the environment.
import os
import sys
os.environ['PYTHONPATH']+=":/content/drive/My Drive/TensorFlow/models"
sys.path.append("/content/drive/My Drive/TensorFlow/models/research")

In [ ]:
#Step 9- Build and Install setup.py.

!python setup.py build
!python setup.py install

In [ ]:
#Step 10- Test the installation.

#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

# note
(You should have the images in test and train folder (with their corresponding XML files) and label_map.pbtxt file ready in respective directories.

You should also have the generate_tfrecord.py in your preprocessing directory.

If you don't have these files ready, go back to Step 1 and finish downloading required files.)

Bạn nên có các hình ảnh trong thư mục thử nghiệm và huấn luyện (với các tệp XML tương ứng của chúng) và tệp label_map.pbtxt đã sẵn sàng trong các thư mục tương ứng.

Bạn cũng nên có create_tfrecord.py trong thư mục tiền xử lý của mình.

Nếu bạn chưa chuẩn bị sẵn các tệp này, hãy quay lại Bước 1 và hoàn tất việc tải xuống các tệp cần thiết.

In [ ]:

#Step 11- Generate TFrecords.

#cd into preprocessing directory
%cd '/content/drive/My Drive/TensorFlow/scripts/preprocessing'

#run the cell to generate test.record and train.record
!python generate_tfrecord.py -x '/content/drive/My Drive/TensorFlow/workspace/training_demo/images/train' -l '/content/drive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/train.record'
!python generate_tfrecord.py -x '/content/drive/My Drive/TensorFlow/workspace/training_demo/images/test' -l '/content/drive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/test.record'

# !python generate_tfrecord.py -x '[path_to_train_folder]' -l '[path_to_annotations_folder]/label_map.pbtxt' -o '[path_to_annotations_folder]/train.record'
# !python generate_tfrecord.py -x '[path_to_test_folder]' -l '[path_to_annotations_folder]/label_map.pbtxt' -o '[path_to_annotations_folder]/test.record'

NOTE:

If you haven't downloaded any pre-trained model yet, go back to Step 1 and finish downloading any pre-trained model of your choice.

We are almost ready to start our model training, just a few more steps before we start our model training!

Step 12- Copying some files


*   Copy the "model_main_tf2.py" file from   "TensorFlow\models\research\object_detection" and paste it into training_demo. We will need this file for training the model.

*   Copy the "exporter_main_v2.py" file from "TensorFlow\models\research\object_detection" and paste it into training_demo.
We will need this file to export the trained model

Step 13- Configure the pipeline file.

Refer the mentioned Medium article for more details!

In [ ]:
#Step 14- Start Tensorboard.

#cd into training_demo
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

#start the Tensorboard
%load_ext tensorboard
%tensorboard --logdir=models/my_ssd_resnet50_v1_fpn

# %load_ext tensorboard
# %tensorboard --logdir=models/[name_of_pre-trained-model_you_downloaded]

In [ ]:
#optional
#code to check how much session time is remaining 

import time,psutil
uptime=time.time()-psutil.boot_time()
remaintime=(12*60*60)-uptime
print(remaintime/(60*60))

In [ ]:
#Step 15- Train the model.

#run the cell to start model training 
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

# !python model_main_tf2.py --model_dir=models/[name_of_pre-trained-model_you_downloaded] --pipeline_config_path=models/[name_of_pre-trained-model_you_downloaded]/pipeline.config

Congratulations! You have finished model training!

In [ ]:
#Step 16- Export the Trained Model.

#run the cell to start model training 
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn/ --output_directory ./exported-models/my_model

# !python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/[name_of_pre-trained-model you downloaded]/pipeline.config --trained_checkpoint_dir ./models/[name_of_pre-trained-model_you_downloaded]/ --output_directory ./exported-models/my_model

We have finished training and exporting our model. It's time to test our model!

In [ ]:
#Step 17- Test the Model.

#Loading the saved_model
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL="/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/my_model/saved_model"

print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')

In [ ]:
#Step 18- Testing the Model.

#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt",use_display_name=True)

#category_index=label_map_util.create_category_index_from_labelmap([path_to_label_map],use_display_name=True)

In [ ]:
#Step 19- Testing the Model.

#Loading the image
img=['/content/img1.jpg','/content/img2.jpg']
print(img)

#list containing paths of all the images

In [ ]:
#Step 20- Running the Inference.

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

for image_path in img:

    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()
    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor=tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor=input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections=detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}
    detections['num_detections']=num_detections

    # detection_classes should be ints.
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    image_np_with_detections=image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=1,     #max number of bounding boxes in the image
          min_score_thresh=.3,      #min prediction threshold
          agnostic_mode=False)
    %matplotlib inline
    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
    plt.show()